### Phys 629: Statistical Tools for Physics Research
***Anuradha Gupta***

# Homework 5
### Due: Friday, Oct 6 at 11:59 pm CT

## Problem 1

This week we have only one problem worth 20 points. This problem uses a dataset in `/coursework/homeworks/hw_data/`.

1) Read in `hw5_data.npy`. This is a (50 x 2) numpy array, with measurements in the first column and uncertainties in the second column. Using the analytic results for heteroscedastic Gaussian data from lectures, compute the sample mean and the standard error on the sample mean from for this data.

2) Reusing some approaches and tools from `lecture_11`, write a ln-likelihood function for heteroscedastic Gaussian data, and use it in a fitting algorithm to find the best-fit mean. *Remember that scipy optimizers are set up to minimize functions.*

3) Using the same numerical technique from `lecture_10`, compute the Fisher uncertainty estimate on the mean.

4) While we have fitted a heteroscedastic Gaussian to this data, let's try something else. Write some code to define a ln-likelihood for a Laplace distribution evaluated on this data. Fit simultaneously for the Laplace location parameter $\mu$ and scale parameter $\Delta$.

5) Compute the AIC values for the heteroscedastic Gaussian model and the Laplacian model. Which model is favored by the data?

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Load the .npy file
data = np.load('hw5_data.npy')

# Create a DataFrame and specify column names
df = pd.DataFrame(data, columns=['measurements', 'uncertainties'])

# Now, 'df' is a DataFrame with named columns




In [33]:
df

,measurements,uncertainties
0,2.972077,0.938065
1,1.988243,1.402627
2,1.669820,1.971207
3,3.965197,0.603593
4,3.385415,1.296987
...,...,...
95,4.077840,1.442946
96,4.101358,1.939565
97,3.080222,1.012284
98,4.536944,1.923794


<h1> 1.1 Computing the sample mean and the standard error on the sample mean </h1>

In [34]:
#Extract the 'measurements' and 'uncertainties' columns

measurements = df['measurements']
uncertainties = df['uncertainties']

# Calculate the MLE for the mean (mu_hat)
mu_hat = np.sum(measurements / uncertainties**2) / np.sum(1 / uncertainties**2)

# Calculate the uncertainty of the mean (sigma_mu)
sigma_mu = 1 / np.sqrt(np.sum(1 / uncertainties**2))

print(f"MLE for mean (mu_hat): {mu_hat:.9f}")
print(f"Uncertainty of the mean (sigma_mu): {sigma_mu:.9f}")

MLE for mean (mu_hat): 3.917992035
Uncertainty of the mean (sigma_mu): 0.094810841


<h1> 1.2 Best fit $\mu$ </h1>

In [35]:
import scipy.optimize as opt

# Define the negative log-likelihood function
def neg_log_likelihood(mu, x, sigma):
    residuals = (x - mu) ** 2 / sigma**2 + np.log(sigma**2)
    return 0.5 * np.sum(residuals)

# Initial guess for the mean (mu)
initial_guess = np.mean(measurements)

# Minimize the negative log-likelihood to find the best-fit mean (mu)
result = opt.minimize(neg_log_likelihood, initial_guess, args=(measurements, uncertainties))

# Extract the best-fit mean from the result
best_fit_mean = result.x[0]

print(f"Best-fit mean (mu): {best_fit_mean:.9f}")


Best-fit mean (mu): 3.917992028


<h1> 1.3 Fisher uncertainty estimate </h1>

In [39]:

# Compute the Fisher Information Matrix (FIM)
def compute_fisher_information(mu, x, sigma):
    n = len(x)
    FIM = np.zeros((1, 1))  # Initialize a 1x1 matrix for the Fisher Information Matrix
    for i in range(n):
        term = 1 / sigma[i]**2
        FIM += term
    return FIM

FIM = compute_fisher_information(best_fit_mean, measurements, uncertainties)

# Compute the Fisher uncertainty estimate on the mean (sigma_mu)
sigma_mu = np.sqrt(np.linalg.inv(FIM)[0, 0])


print(f"Fisher uncertainty estimate on the mean (sigma_mu): {sigma_mu:.9f}")


Fisher uncertainty estimate on the mean (sigma_mu): 0.094810841


<h1> 1.4 Laplace Log-Likelihood </h1>

In [42]:


# Define the negative log-likelihood function for Laplace distribution
def neg_log_likelihood(params, x):
    mu, delta = params
    n = len(x)
    log_likelihood = -n * np.log(2 * delta) - np.sum(np.abs(x - mu) / delta)
    return -log_likelihood  # Return negative log-likelihood for minimization



# Initial guesses for parameters (mu and delta)
initial_guess = [np.mean(measurements), np.std(measurements)]

# Minimize the negative log-likelihood to find MLEs of parameters (mu and delta)
result = opt.minimize(neg_log_likelihood, initial_guess, args=(measurements,))

# Extract the MLEs of parameters (mu and delta) from the result
mu_mle, delta_mle = result.x

print(f"MLE for Laplace location parameter (mu): {mu_mle:.9f}")
print(f"MLE for Laplace scale parameter (delta): {delta_mle:.9f}")


MLE for Laplace location parameter (mu): 4.085951634
MLE for Laplace scale parameter (delta): 0.882269239


<h1> 1.5 </h1>

In [43]:

# Define the negative log-likelihood function for the Laplace distribution
def neg_log_likelihood_laplace(params, x):
    mu, delta = params
    n = len(x)
    log_likelihood = -n * np.log(2 * delta) - np.sum(np.abs(x - mu) / delta)
    return -log_likelihood  # Return negative log-likelihood for minimization


# Define the negative log-likelihood function for heteroscedastic Gaussian distribution
def neg_log_likelihood_gaussian(params, x, sigma):
    mu, = params
    residuals = (x - mu) ** 2 / sigma**2 + np.log(sigma**2)
    return 0.5 * np.sum(residuals)



# Initial guesses for parameters (mu and delta) for Laplace model
initial_guess_laplace = [np.mean(measurements), np.std(measurements)]

# Initial guess for parameter (mu) for Gaussian model
initial_guess_gaussian = [np.mean(measurements)]



# Minimize the negative log-likelihood to find MLEs of parameters for Laplace model
result_laplace = opt.minimize(neg_log_likelihood_laplace, initial_guess_laplace, args=(measurements,))

# Minimize the negative log-likelihood to find MLEs of parameter for Gaussian model
result_gaussian = opt.minimize(neg_log_likelihood_gaussian, initial_guess_gaussian, args=(measurements, uncertainties))



# Extract the MLEs of parameters for Laplace model (mu and delta) from the result
mu_mle_laplace, delta_mle = result_laplace.x

# Extract the MLE of parameter for Gaussian model (mu) from the result
mu_mle_gaussian = result_gaussian.x[0]



# Calculate the maximum log-likelihood for both models
max_log_likelihood_laplace = -neg_log_likelihood_laplace(result_laplace.x, measurements)
max_log_likelihood_gaussian = -neg_log_likelihood_gaussian(result_gaussian.x, measurements, uncertainties)



# Calculate the number of parameters for each model
k_laplace = 2  # Laplace model has two parameters (mu and delta)
k_gaussian = 1  # Gaussian model has one parameter (mu)



# Calculate the AIC values for both models
N = len(measurements)
aic_laplace = -2 * max_log_likelihood_laplace + 2 * k_laplace + (2 * k_laplace * (k_laplace + 1)) / (N - k_laplace - 1)
aic_gaussian = -2 * max_log_likelihood_gaussian + 2 * k_gaussian + (2 * k_gaussian * (k_gaussian + 1)) / (N - k_gaussian - 1)

print(f"AIC for Laplace Model: {aic_laplace:.4f}")
print(f"AIC for Gaussian Model: {aic_gaussian:.4f}")



# Compare AIC values and determine which model is favored
if aic_laplace < aic_gaussian:
    print("The Laplace model is favored by the data.")
else:
    print("The Gaussian model is favored by the data.")


AIC for Laplace Model: 317.7015
AIC for Gaussian Model: 112.7688
The Gaussian model is favored by the data.
